# 15 P2 Project

* ist1102903 Ana Alfaiate (33%)
  
* ist1102902 Maria Inês Trigueiro (33%)
  
* ist1102881 Raquel Coelho (33%)

Prof. Flávio Martins (and Prof. João Marques)

Lab Shift number: PB03

## PART I – Original Database Schema (SQLite)

##### E-R Diagram

![E-R Diagram](soccer.drawio.png "E-R Diagram")

#### 1. The Original Database Schema

Study the E-R Diagram above corresponding to the original database schema.


Lets download the SQLite database backup/dump!

In [1]:
!wget -c https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download= -O /home/jovyan/data/database.sqlite

-
-
2
0
2
4
-
1
2
-
2
8
 
1
7
:
5
7
:
3
4
-
-
 
 
h
t
t
p
s
:
/
/
g
i
t
h
u
b
.
c
o
m
/
b
d
i
s
t
/
i
s
-
l
a
b
s
-
d
a
t
a
/
r
a
w
/
r
e
f
s
/
h
e
a
d
s
/
m
a
i
n
/
s
o
c
c
e
r
/
d
a
t
a
b
a
s
e
.
s
q
l
i
t
e
?
d
o
w
n
l
o
a
d
=

R
e
s
o
l
v
i
n
g
 
g
i
t
h
u
b
.
c
o
m
 
(
g
i
t
h
u
b
.
c
o
m
)
.
.
.
 
1
4
0
.
8
2
.
1
2
1
.
4

C
o
n
n
e
c
t
i
n
g
 
t
o
 
g
i
t
h
u
b
.
c
o
m
 
(
g
i
t
h
u
b
.
c
o
m
)
|
1
4
0
.
8
2
.
1
2
1
.
4
|
:
4
4
3
.
.
.
 
c
o
n
n
e
c
t
e
d
.

H
T
T
P
 
r
e
q
u
e
s
t
 
s
e
n
t
,
 
a
w
a
i
t
i
n
g
 
r
e
s
p
o
n
s
e
.
.
.
 
3
0
2
 
F
o
u
n
d

L
o
c
a
t
i
o
n
:
 
h
t
t
p
s
:
/
/
m
e
d
i
a
.
g
i
t
h
u
b
u
s
e
r
c
o
n
t
e
n
t
.
c
o
m
/
m
e
d
i
a
/
b
d
i
s
t
/
i
s
-
l
a
b
s
-
d
a
t
a
/
r
e
f
s
/
h
e
a
d
s
/
m
a
i
n
/
s
o
c
c
e
r
/
d
a
t
a
b
a
s
e
.
s
q
l
i
t
e
?
d
o
w
n
l
o
a
d
=
t
r
u
e
 
[
f
o
l
l
o
w
i
n
g
]

-
-
2
0
2
4
-
1
2
-
2
8
 
1
7
:
5
7
:
3
5
-
-
 
 
h
t
t
p
s
:
/
/
m
e
d
i
a
.
g
i
t
h
u
b
u
s
e
r
c
o
n
t
e
n
t
.
c
o
m
/
m
e
d
i
a
/
b
d
i
s
t
/
i
s
-
l
a
b
s
-

#### 2. SQL Queries

Consider the E-R Diagram above and write SQL queries for each of the following information needs.

In [4]:
%reload_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql sqlite:////home/jovyan/data/database.sqlite --alias sqlite

1. What is the maximum number of goals recorded in a match? Output a single number.

In [14]:
%%sql sqlite

--select match_api_id, max(num_match_goal)
select max(num_match_goal)
from (
    SELECT match_api_id,  home_team_goal + away_team_goal as num_match_goal
    FROM match
    Group by match_api_id);

max(num_match_goal)
12


2. List each league name and its number of teams sorted from high to low.

In [21]:
%%sql sqlite

Select l.id, l.name, count(distinct team_api_id) as nr_teams
from league l , match, team t
where l.id = league_id and (t.team_api_id = home_team_api_id OR t.team_api_id = away_team_api_id)
group by l.name
order by nr_teams desc

id,name,nr_teams
4769,France Ligue 1,35
1729,England Premier League,34
21518,Spain LIGA BBVA,33
10257,Italy Serie A,32
7809,Germany 1. Bundesliga,30
17642,Portugal Liga ZON Sagres,29
13274,Netherlands Eredivisie,25
1,Belgium Jupiler League,24
15722,Poland Ekstraklasa,22
19694,Scotland Premier League,17


3. Who are the GoalKeepers (GK) that have played all the away games for their team? Output the `team_long_name` and `player_name` (assuming `match.away_player_1` is the goalkeeper column).

In [55]:
%%sql sqlite

select c.away_player_1, player_name
from (
    select away_team_api_id, away_player_1, count(away_team_api_id) as games_gk --, sum(count(*)) OVER (PARTITION BY away_team_api_id) AS total_games_team
    from match
    group by away_team_api_id, away_player_1
    order by away_team_api_id) c

    JOIN (
    SELECT 
        m.away_team_api_id,
        COUNT(*) AS total_games_team  -- Total games per team
    FROM 
        match m
    GROUP BY 
        m.away_team_api_id
) g ON c.away_team_api_id = g.away_team_api_id, player 
where c.games_gk = g.total_games_team and c.away_player_1 = player.player_api_id


--select away_team_api_id, count(away_team_api_id)
--from match
--group by away_team_api_id
--select away_team_api_id, match.away_player_1
--    team.id, team_long_name, match.away_player_1
--from match
--where match.away_team_api_id = team.id

away_player_1,player_name
114963,Fabian Giefer
448805,Christian Mathenia
33339,Gerhard Tremmel
37335,Lukas Kruse
33958,Asier Riesgo
30455,Heurelho Gomes
279175,Baptiste Reynet
2984,Sergio Aragones
36058,Dennis Eilhoff


## PART II - New Database Schema (PostgreSQL)

#### 0. Migrate the SQLite Database to PostgreSQL

Instead of writing tedious migration code ourselves, we will be using the `pgloader` CLI tool to get started quickly. The migration task is setup using a `command-file`. Lets see inside that file.

In [68]:
!cat ./command-file.txt

l
o
a
d
 
d
a
t
a
b
a
s
e

 
 
 
 
f
r
o
m
 
'
/
h
o
m
e
/
j
o
v
y
a
n
/
d
a
t
a
/
d
a
t
a
b
a
s
e
.
s
q
l
i
t
e
'

 
 
 
 
i
n
t
o
 
p
o
s
t
g
r
e
s
q
l
:
/
/
d
b
:
d
b
@
p
o
s
t
g
r
e
s
/
d
b


w
i
t
h
 
i
n
c
l
u
d
e
 
d
r
o
p
,
 
c
r
e
a
t
e
 
t
a
b
l
e
s
,
 
c
r
e
a
t
e
 
i
n
d
e
x
e
s
,
 
r
e
s
e
t
 
s
e
q
u
e
n
c
e
s
,


p
r
e
f
e
t
c
h
 
r
o
w
s
 
=
 
1
0
0
,
 
m
a
x
 
p
a
r
a
l
l
e
l
 
c
r
e
a
t
e
 
i
n
d
e
x
 
=
 
1


s
e
t
 
w
o
r
k
_
m
e
m
 
t
o
 
'
1
6
M
B
'
,
 
m
a
i
n
t
e
n
a
n
c
e
_
w
o
r
k
_
m
e
m
 
t
o
 
'
5
1
2
 
M
B
'
;



In [67]:
%%sql
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

table_name
radiography
surgical
associated
diagnostic_code
prescription
medication
procedure
performs
other_procedures
test


The `pgloader` tool will take care of everything automatically including the foreign key constraints (which SQLite does not enforce by default). Lets run it.

In [69]:
!pgloader ./command-file.txt

/
b
i
n
/
b
a
s
h
:
 
l
i
n
e
 
1
:
 
p
g
l
o
a
d
e
r
:
 
c
o
m
m
a
n
d
 
n
o
t
 
f
o
u
n
d



Closing sqlite

Closing psql

Lets take a closer look into the `match` table.

In [58]:
%load_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql postgresql+psycopg://db:db@postgres/db --alias psql

T
h
e
 
s
q
l
 
e
x
t
e
n
s
i
o
n
 
i
s
 
a
l
r
e
a
d
y
 
l
o
a
d
e
d
.
 
T
o
 
r
e
l
o
a
d
 
i
t
,
 
u
s
e
:

 
 
%
r
e
l
o
a
d
_
e
x
t
 
s
q
l



In [60]:
%reload_ext sql

In [65]:
%sqlcmd columns -t match

T
a
b
l
e
N
o
t
F
o
u
n
d
E
r
r
o
r
:
 
T
h
e
r
e
 
i
s
 
n
o
 
t
a
b
l
e
 
w
i
t
h
 
n
a
m
e
 
'
m
a
t
c
h
'
 
i
n
 
t
h
e
 
d
e
f
a
u
l
t
 
s
c
h
e
m
a

I
f
 
y
o
u
 
n
e
e
d
 
h
e
l
p
 
s
o
l
v
i
n
g
 
t
h
i
s
 
i
s
s
u
e
,
 
s
e
n
d
 
u
s
 
a
 
m
e
s
s
a
g
e
:
 
h
t
t
p
s
:
/
/
p
l
o
o
m
b
e
r
.
i
o
/
c
o
m
m
u
n
i
t
y



Notice how `pgloader` even assigned the safest most-compatible data types for each column so that no data is lost in the migration.

##### 0.1. Introducing a limitation of the original database schema

The `match` table still does not look good to us... Can you try to write a SQL query for the following information need?

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [ ]:
%%sql psql

SELECT DISTINCT t.team_name
FROM match m
JOIN player p ON (
    p.player_api_id = m.home_player_1 OR
    p.player_api_id = m.home_player_2 OR
    p.player_api_id = m.home_player_3 OR
    p.player_api_id = m.home_player_4 OR
    p.player_api_id = m.home_player_5 OR
    p.player_api_id = m.home_player_6 OR
    p.player_api_id = m.home_player_7 OR
    p.player_api_id = m.home_player_8 OR
    p.player_api_id = m.home_player_9 OR
    p.player_api_id = m.home_player_10 OR
    p.player_api_id = m.home_player_11 OR
    p.player_api_id = m.away_player_1 OR
    p.player_api_id = m.away_player_2 OR
    p.player_api_id = m.away_player_3 OR
    p.player_api_id = m.away_player_4 OR
    p.player_api_id = m.away_player_5 OR
    p.player_api_id = m.away_player_6 OR
    p.player_api_id = m.away_player_7 OR
    p.player_api_id = m.away_player_8 OR
    p.player_api_id = m.away_player_9 OR
    p.player_api_id = m.away_player_10 OR
    p.player_api_id = m.away_player_11
)
JOIN team t ON (
    t.team_api_id = m.home_team_api_id OR
    t.team_api_id = m.away_team_api_id
)
WHERE p.player_name = 'Cristiano Ronaldo';


-- WILL NOT BE EVALUATED - We will be implementing a solution for this problem in the next section.

#### 1. Functions, Stored Procedures, and Triggers

Provide the SQL instructions corresponding to each of the following tasks:

1. Create a new `match_player` table to store the data for the starting players (11) for each match (currently stored in the columns `match.home_player_DD` and `match.away_player_DD`). We are going to copy the data contained in the table `match` to `match_player` but we want to use a new column for the player position `position`.

In [ ]:
%%sql psql
DROP TABLE IF EXISTS match_player;

CREATE TABLE
  match_player (
    match_id INT NOT NULL,
    team_id INT NOT NULL,
    player_id INT NOT NULL,
    position INT NOT NULL,
    PRIMARY KEY (match_id, player_id), -- Composite Primary Key
    FOREIGN KEY (match_id) REFERENCES match(id),
    FOREIGN KEY (team_id) REFERENCES team(team_api_id),
    FOREIGN KEY (player_id) REFERENCES player(player_api_id)
);

2. Populate the new table `match_player` using the data contained in the table `match` using a function or stored procedure.

In [ ]:
%%sql psql
DELETE FROM match_player;

CREATE OR REPLACE PROCEDURE populate_match_player()
LANGUAGE plpgsql
AS $$
DECLARE
    match_row match%ROWTYPE;
    position INT;
BEGIN
    FOR match_row IN SELECT * FROM match LOOP
        FOR position IN 1..11 LOOP
            EXECUTE format(
                'INSERT INTO match_player (match_id, team_id, player_id, position)
                 VALUES (%s, %s, %s, %s)',
                match_row.id,
                match_row.home_team_api_id,
                (CASE position
                     WHEN 1 THEN match_row.home_player_1
                     WHEN 2 THEN match_row.home_player_2
                     WHEN 3 THEN match_row.home_player_3
                     WHEN 4 THEN match_row.home_player_4
                     WHEN 5 THEN match_row.home_player_5
                     WHEN 6 THEN match_row.home_player_6
                     WHEN 7 THEN match_row.home_player_7
                     WHEN 8 THEN match_row.home_player_8
                     WHEN 9 THEN match_row.home_player_9
                     WHEN 10 THEN match_row.home_player_10
                     WHEN 11 THEN match_row.home_player_11
                 END),
                position
            );
        END LOOP;

        -- Insert away players
        FOR position IN 1..11 LOOP
            EXECUTE format(
                'INSERT INTO match_player (match_id, team_id, player_id, position)
                 VALUES (%s, %s, %s, %s)',
                match_row.id,
                match_row.away_team_api_id,
                (CASE position
                     WHEN 1 THEN match_row.away_player_1
                     WHEN 2 THEN match_row.away_player_2
                     WHEN 3 THEN match_row.away_player_3
                     WHEN 4 THEN match_row.away_player_4
                     WHEN 5 THEN match_row.away_player_5
                     WHEN 6 THEN match_row.away_player_6
                     WHEN 7 THEN match_row.away_player_7
                     WHEN 8 THEN match_row.away_player_8
                     WHEN 9 THEN match_row.away_player_9
                     WHEN 10 THEN match_row.away_player_10
                     WHEN 11 THEN match_row.away_player_11
                 END),
                position
            );
        END LOOP;
    END LOOP;
END;
END;
$$;

CALL populate_match_player();

#### 2. SQL Queries

Consider the new `match_player` table above and write SQL queries for each of the following information needs.

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [ ]:
%%sql psql

select team_id
from match_player mp, player p
where mp.player_id = p.player_id and p.player_name = 'Cristiano Ronaldo'

2. Who are the players that have played all the away games for their team? Output the `team_long_name` and `player_name`.

In [ ]:
%%sql psql

select mp.player_id, player_name
from (
    select away_team_api_id, player_id, count(away_team_api_id) as games_player
    from match, match_player mp
    where mp.team_id = away_team_api_id

    group by away_team_api_id, player_id
    order by away_team_api_id) gp

    JOIN (
    SELECT 
        m.away_team_api_id,
        COUNT(*) AS total_games_team  
    FROM 
        match m
    GROUP BY 
        m.away_team_api_id
) gt ON gp.away_team_api_id = gt.away_team_api_id, player 
where c.games_player = g.total_games_team and gp.away_player_1 = player.player_api_id

3. What is the match with the highest number of goals? Output `team_long_name` for both home and away teams, and the total number of goals for each team.

In [ ]:
%%sql psql

select match_api_id, h.team_long_name as home_team, a.team_long_name as away_team
from (select match_api_id, home_team_goal + away_team_goal as num_match_goal
    from match
    group by match_api_id) --, team h, team a
JOIN team h ON m.home_team_api_id = h.team_api_id
JOIN team a ON m.away_team_api_id = a.team_api_id
where num_match_goal > ALL  (
    SELECT home_team_goal + away_team_goal AS num_match_goal
    FROM match
    GROUP BY match_api_id
); --AND home_team_api_id = h.team_api_id AND away_team_api_id = a.team_long_name;

#### 3. Query Optimization

Consider the new `player_attributes` table that contains the largest amount of rows.

1. Create the most appropriate index(es) for the following query: 

In [ ]:
%%sql psql
CREATE INDEX

In [ ]:
%%sql psql
SELECT
  MAX(overall_rating)
FROM
  player_attributes
WHERE
  attacking_work_rate = 'high';

2. Create the most appropriate index(es) for the following query: 

In [ ]:
%%sql psql
CREATE INDEX

In [ ]:
%%sql psql
SELECT
  player_api_id,
  preferred_foot,
  MIN(crossing),
  MIN(potential)
FROM
  player_attributes
GROUP BY
  player_api_id,
  preferred_foot
HAVING
  MIN(crossing) = 80
  AND MIN(potential) >= 90

## PART III - The Web App (Flask + PostgreSQL)

#### 1. SQL View

1. Create a SQL View that lists the player names sorted by the most recent `player_attributes.date` first.

In [ ]:
%%sql psql
CREATE OR REPLACE VIEW
  players_index_view

2. Create the most appropriate index(es) for the SQL view you just created. (Note: PostgreSQL does not have support for materialized views)

In [ ]:
%%sql psql
CREATE INDEX

3. Please test your view using the following query.

In [ ]:
%%sql psql
SELECT
  *
FROM
  players_index_view
LIMIT
  20;

#### 2. Create the players_index page on the Web App

Requirements:
- Should use the SQL View declared above
- Do not list more than 20 players.

1. Copy your players_index function from app.py to the cell below

```python
@app.route("/", methods=("GET",))
@app.route("/accounts", methods=("GET",))
@limiter.limit("1 per second")
def account_index():
    """Show all the accounts, most recent first."""

    with pool.connection() as conn:
        with conn.cursor() as cur:
            accounts = cur.execute(
                """
                SELECT account_number, branch_name, balance
                FROM account
                ORDER BY account_number DESC;
                """,
                {},
            ).fetchall()
            log.debug(f"Found {cur.rowcount} rows.")

    return render_template("account/index.html", accounts=accounts)
```

2. Copy your final HTML template for this view onto the cell below

```html
{% extends "base.html" %}
{% block header %}
    <h1>
        {% block title %}
            Accounts
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}
    {% for account in accounts %}
        <article class="post">
            <header>
                <div>
                    <h1>{{ account['account_number'] }}</h1>
                    <div class="about">in {{ account['branch_name'] }}</div>
                </div>
                <a class="action"
                   href="{{ url_for('account_update_view', account_number=account['account_number']) }}">Edit</a>
            </header>
            <p class="body">€ {{ account['balance'] }}</p>
        </article>
        {% if not loop.last %}<hr />{% endif %}
    {% endfor %}
{% endblock content %}
```

#### 3. Create the player_update_view and player_update_save routes on the Web App

Requirements:

- The app.route for player_update_view should take a player_api_id and show a nice editable HTML form.
             
- The app.route for player_update_save should be called on form submission and insert the new data into player_attributes. 

- On success, should redirect to the players_index

- On failure, should raise an appropriate exception with a error message

1. Copy your player_update_view function from app.py to the cell below

2. Copy your final HTML template for this view onto the cell below

3. Copy your player_update_save function from app.py to the cell below